# TensorFlow Lite

## TinyML Application Pipeline

![](fig/tf-lite/pipeline.png)

- Step 1: Collect & Preprocess Data
- Step 2: Design and Train a Model
- Step 3: Evaluate, Optimize, Convert and Deploy Model

Underneath the models are the ML runtimes:

- TensorFlow
- TensorFlow Lite
- TensorFlow Lite Micro

## Model Training vs. Deployment

- Left sides: architecture and APIs that can be used for training models
- Right sides: how the models can be deployed
    - The standard TensorFlow models
    - TensorFlow Lite runtime
    - TensorFlow Lite Micro, built on top of TF Lite and can be used to shrink you model further. 
    - TensorFlow.js provides a Javascript-based library for training models and running inference on them.


In [2]:
import tensorflow as tf
print (tf.__version__)

2.18.0


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(units = 1, input_shape=[1])
])

model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))

In [21]:
export_dir = 'data/model1_savedmodel'
model.export(export_dir)

INFO:tensorflow:Assets written to: data/model1_savedmodel\assets


INFO:tensorflow:Assets written to: data/model1_savedmodel\assets


Saved artifact at 'data/model1_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_6')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2102073780240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102073780624: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Saved Model Conversion

- With the saved model, we can convert the model into the TFLite format

In [22]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [23]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1116

In [24]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor_6:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [25]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.981907]]
